### Nome: Cristiano Moreira Martins
### Matrícula: 11111871-7

### 1. Escolher um conjunto de dados público para realização da tarefa. Algumas sugestões de datasets que podem ser utilizados são: Penguins, Iris, AMES House Prices.

Foram utilizados os dados disponibilizados na biblioteca <code>pandas-datareader</code> (https://pydata.github.io/pandas-datareader/readers/index.html). 

Os dados escolhidos são as informações de instrumentos financeiros listados na NASDAQ que a biblioteca disponibiliza: 
https://pydata.github.io/pandas-datareader/readers/nasdaq-trader.html

Ao todo são carregados 11550 instrumentos.

### 2. Criar uma API Rest que permita: ...

Para o desenvolvimento da API, foi criado um arquivo python <code>app.py</code> para comportar a API e os métodos solicitados. 
- Foi implementada a paginação dos dados no método GET, quando solicitado todos os instrumentos.
- As colunas originais da fonte de dados foram renomeadas, na função <code>load_data()</code>, por comodidade.
- Os dados permanecem em memória, em um dicionário.
- Para rodar Api, digitar no terminal onde estão os arquivos do projeto: <code>python app.py</code>
- A biblioteca busca os dados, no entanto foi realizada uma cópia dos dados para um arquivo csv localizado na pasta <code>data</code>, para ser utilizado caso não seja possível carregar os dados pela biblioteca.

In [ ]:
# Instala as bibliotecas necessárias...
!pip install flask pandas pandas-datareader requests

### 3. Consumir a API criada usando a lib requests. Explorar todas as operações da API

In [19]:
import requests
import pandas as pd

##### a) Consulta - Método HTTP GET;

In [20]:
base_url = 'http://localhost:5000/api/symbols'

response = requests.get(base_url)
pd.DataFrame(response.json())

,NASDAQSymbol,category,cqsSymbol,exchange,index,isETF,isTest,isTraded,lotSize,name,nextShares,status,symbol
0,A,,A,N,0,False,False,True,100.0,"Agilent Technologies, Inc. Common Stock",False,NaN,A
1,AA,,AA,N,1,False,False,True,100.0,Alcoa Corporation Common Stock,False,NaN,AA
2,AAA,,AAA,P,2,True,False,True,100.0,Investment Managers Series Trust II AXS First ...,False,NaN,AAA
3,AAAU,,AAAU,Z,3,True,False,True,100.0,Goldman Sachs Physical Gold ETF Shares,False,NaN,AAAU
4,AAC,,AAC,N,4,False,False,True,100.0,Ares Acquisition Corporation Class A Ordinary ...,False,NaN,AAC
5,AAC=,,AAC.U,N,5,False,False,True,100.0,"Ares Acquisition Corporation Units, each consi...",False,NaN,AAC.U
6,AAC+,,AAC.WS,N,6,False,False,True,100.0,Ares Acquisition Corporation Redeemable Warran...,False,NaN,AAC.W
7,AACG,G,NaN,Q,7,False,False,True,100.0,ATA Creativity Global - American Depositary Sh...,False,N,AACG
8,AACI,G,NaN,Q,8,False,False,True,100.0,Armada Acquisition Corp. I - Common Stock,False,N,AACI
9,AACIU,G,NaN,Q,9,False,False,True,100.0,Armada Acquisition Corp. I - Unit,False,N,AACIU


In [21]:
# Consulta - Método HTTP GET, com parâmetros de paginação na URL
base_url = 'http://localhost:5000/api/symbols?page=6&per_page=5'

response = requests.get(base_url)
pd.DataFrame(response.json())

,NASDAQSymbol,category,cqsSymbol,exchange,index,isETF,isTest,isTraded,lotSize,name,nextShares,status,symbol
0,AAON,Q,NaN,Q,25,False,False,True,100.0,"AAON, Inc. - Common Stock",False,N,AAON
1,AAP,,AAP,N,26,False,False,True,100.0,Advance Auto Parts Inc.,False,NaN,AAP
2,AAPB,G,NaN,Q,27,True,False,True,100.0,GraniteShares 1.75x Long AAPL Daily ETF,False,N,AAPB
3,AAPD,G,NaN,Q,28,True,False,True,100.0,Direxion Daily AAPL Bear 1X Shares,False,N,AAPD
4,AAPL,Q,NaN,Q,29,False,False,True,100.0,Apple Inc. - Common Stock,False,N,AAPL


In [22]:
# Consulta - Método HTTP GET (por símbolo)

base_url = 'http://localhost:5000/api/symbols/AAPL'
response = requests.get(base_url)
pd.DataFrame([response.json()])

,NASDAQSymbol,category,cqsSymbol,exchange,index,isETF,isTest,isTraded,lotSize,name,nextShares,status,symbol
0,AAPL,Q,NaN,Q,29,False,False,True,100.0,Apple Inc. - Common Stock,False,N,AAPL


##### b) Atualização - Método HTTP PUT;

In [23]:
base_url = 'http://localhost:5000/api/symbols'

symbol = 'AAPL'
response = requests.get(f'{base_url}/{symbol}')
aapl = pd.DataFrame([response.json()])

# Altera o valor de alguns campos
aapl['isTraded'] = False
aapl['name'] = 'Apple Inc.'
aapl['lotSize'] = 150.0
aapl['cqsSymbol'] = None

# Envia alterações para o servidor
aapl = aapl.to_dict(orient='records')[0]
response = requests.put(f'{base_url}/{symbol}', json=aapl)
print(response.json())

{'message': 'Instrumento atualizado.'}


In [24]:
# Consulta o símbolo que foi atualizado
response = requests.get(f'{base_url}/AAPL')
pd.DataFrame([response.json()])

,NASDAQSymbol,category,cqsSymbol,exchange,index,isETF,isTest,isTraded,lotSize,name,nextShares,status,symbol
0,AAPL,Q,None,Q,29,False,False,False,150.0,Apple Inc.,False,N,AAPL


##### c) Inserção - Método HTTP POST

In [25]:
base_url = 'http://localhost:5000/api/symbols'

new_symbol = {
    'symbol': 'TEST',
    'isTraded': True,
    'name': 'Test Security',
    'exchange': 'N',
    'category': 'G',
    'isETF': False,
    'lotSize': 100,
    'isTest': False,
    'status': 'N',
    'cqsSymbol': 'TEST',
    'nextShares': False
}
response = requests.post(base_url, json=new_symbol)
print(response.json())


{'message': 'Instrumento adicionado.'}


In [26]:
# Consulta o símbolo adicionado
response = requests.get(f'{base_url}/TEST')
pd.DataFrame([response.json()])

,category,cqsSymbol,exchange,isETF,isTest,isTraded,lotSize,name,nextShares,status,symbol
0,G,TEST,N,False,False,True,100,Test Security,False,N,TEST


##### d) Deleção - Método HTTP DELETE.

In [27]:
response = requests.delete(f'{base_url}/TEST')
print(response.json())

{'message': 'Instrumento removido.'}


In [28]:
# Consulta o símbolo que foi removido
response = requests.get(f'{base_url}/TEST')
print(response.json())

{'error': 'Instrumento não encontrado.'}
